In [ ]:
# ---
# title: 08. Portfolio Construction (The Final Build)
# tags: [Strategy, Integration, AI]
# difficulty: Advanced
# ---

import pandas as pd
import numpy as np
import scipy.optimize as sco
import matplotlib.pyplot as plt
from pathlib import Path

# Setup Paths
gold_path = Path("../data/gold")
silver_path = Path("../data/silver")

# 1. Load AI Predictions (Alpha)
pred_file = gold_path / "backtest_predictions.parquet"
df_preds = pd.read_parquet(pred_file)
latest_alpha = df_preds.iloc[-1] # Predictions for TOMORROW

# 2. Load Historical Returns (for Sigma)
returns_file = max(list(silver_path.glob("market_returns_*.parquet")), key=lambda f: f.stat().st_mtime)
df_returns = pd.read_parquet(returns_file)
cov_matrix = df_returns.cov() # Daily Covariance

# The Black-Litterman Idea (Simplified)

Instead of using historical mean returns (which are noisy), we use our **GNN Predictions** as the Expected Returns vector ($\\mu$).

We keep the Historical Covariance ($\Sigma$) as our risk model.

Objective: $Maximize \frac{w^T \mu_{AI}}{\sqrt{w^T \Sigma w}}$

In [ ]:
# Align Data
tickers = latest_alpha.index.tolist()
cov_matrix = cov_matrix.loc[tickers, tickers]
expected_returns = latest_alpha.values

# Optimization Function
def neg_sharpe(weights, exp_ret, cov):
    p_ret = np.dot(weights, exp_ret)
    p_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights)))
    return - p_ret / p_std # Assumes daily Sharpe

# Constraints
num_assets = len(tickers)
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bounds = tuple((0.0, 0.20) for _ in range(num_assets)) # Max 20% concentration per stock
init_guess = num_assets * [1. / num_assets,]

# Solve
result = sco.minimize(neg_sharpe, init_guess, args=(expected_returns, cov_matrix),
                      method='SLSQP', bounds=bounds, constraints=constraints)

opt_weights = pd.Series(result.x, index=tickers)
print("Optimization Success:", result.success)

# Show Portfolio
final_portfolio = opt_weights[opt_weights > 0.01].sort_values(ascending=False)

plt.figure(figsize=(12, 6))
final_portfolio.plot(kind='bar', color='#00ccff')
plt.title('Final AI-Optimized Portfolio Weights')
plt.ylabel('Weight')
plt.show()

print(final_portfolio.head(10))